<a href="https://colab.research.google.com/github/mbewustanley/yt_insights/blob/main/7_experiment_6_lightgbm_detailed_hpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mlflow boto3 awscli optuna imbalanced-learn lightgbm

In [2]:
# mlflow, boto3, awscli, optuna, xgboost, imbalanced-learn

import optuna # a hyperparameter optimization framework for ml
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [5]:
#set mlflow tracking and experiment
mlflow.set_tracking_uri("http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000")
mlflow.set_experiment('LightGBM HP Tuning')

<Experiment: artifact_location='s3://stanley-mlflow-bucket-27/248940192345400758', creation_time=1764445797771, experiment_id='248940192345400758', last_update_time=1764445797771, lifecycle_stage='active', name='LightGBM HP Tuning', tags={}>

In [6]:
# load data
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [7]:
# remap the class labels from [-1,0,1] to [2,0,1]. xgboost does not take negatove values
df['category'] = df['category'].map({-1:2, 0:0, 1:1})

In [8]:
# remove rows where target labels (category) are NAN
df = df.dropna(subset=['category'])

In [9]:
ngram_range = (1,3) #trigram settings
max_features = 1000  # set max features for TF-IDF

In [10]:
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

In [11]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [12]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [13]:
# function to log results in MLFLOW
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        #log model type
        mlflow.set_tag('mlflow.runName', f"Trial_{trial_number}_{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag('experiment_type', "algorithm_comparison")

        # log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        #log accuracy
        accuracy= accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        #log classification report
        classification_rep =classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric_name, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric_name}", value)

        #log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        return accuracy

In [20]:
# optuna objective function for LightGBM
def objective_lightgbm(trial):
    # Hyperparameter space to explore
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    num_leaves = trial.suggest_int('num_leaves', 20, 150)
    min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True)  #L1 regularization
    reg_lambda = trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True)  #L2 regularization

    #log trial parameters
    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'num_leaves': num_leaves,
        'min_child_samples': min_child_samples,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda
        }
    # create LightGBM model
    model = LGBMClassifier(n_estimators=n_estimators,
                           learning_rate=learning_rate,
                           max_depth=max_depth,
                           num_leaves=num_leaves,
                           min_child_samples=min_child_samples,
                           colsample_bytree=colsample_bytree,
                           subsample=subsample,
                           reg_alpha=reg_alpha,
                           reg_lambda=reg_lambda,
                           random_state=42)

    # log each trial as a separate run in mlflow
    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)

    return accuracy

In [16]:
# run optuna for LightGBM, log the best model only, and plot the importance of each parameter
def run_optuna_experiment():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_lightgbm, n_trials=100)  #increased to 100 trials

    #get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'],
                               learning_rate=best_params['learning_rate'],
                               max_depth = best_params['max_depth'],
                               num_leaves = best_params['num_leaves'],
                               min_child_samples = best_params['min_child_samples'],
                               subsample = best_params['subsample'],
                               colsample_bytree = best_params['colsample_bytree'],
                               reg_alpha = best_params['reg_alpha'],
                               reg_lambda = best_params['reg_lambda'],
                               random_state=42)

    #log the nest model with mlflow, and print the classification report
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params, "Best")

    # Plot parameter importance
    optuna.visualization.plot_param_importances(study).show()

    # Plot optimization history
    optuna.visualization.plot_optimization_history(study).show()

In [21]:
run_optuna_experiment()

[I 2025-12-01 12:17:50,896] A new study created in memory with name: no-name-ba43f2a9-1654-4c5a-9843-01365a96ab5e


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.229730 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 98381
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 956
[LightGBM] [Info] Start training from score -1.093552
[LightGBM] [Info] Start training from score -1.104097
[LightGBM] [Info] Start training from score -1.098216
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/12/01 12:18:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/01 12:19:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_0_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000/#/experiments/248940192345400758/runs/af39b2e1d1d84a3cb9b6a6b7379b3b01
🧪 View experiment at: http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000/#/experiments/248940192345400758


[I 2025-12-01 12:19:29,611] Trial 0 finished with value: 0.7903191714225323 and parameters: {'n_estimators': 654, 'learning_rate': 0.024873228038112927, 'max_depth': 8, 'num_leaves': 135, 'min_child_samples': 80, 'subsample': 0.817553904974085, 'colsample_bytree': 0.7765091718321722, 'reg_alpha': 0.004085533940960514, 'reg_lambda': 3.6311755314701157}. Best is trial 0 with value: 0.7903191714225323.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.234192 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 98601
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 966
[LightGBM] [Info] Start training from score -1.093552
[LightGBM] [Info] Start training from score -1.104097
[LightGBM] [Info] Start training from score -1.098216
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🏃 View run Trial_29_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000/#/experiments/248940192345400758/runs/52ccf9065ac3452c955e68cc8d3e74fb
🧪 View experiment at: http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000/#/experiments/248940192345400758


[W 2025-12-01 13:30:10,600] Trial 29 failed with parameters: {'n_estimators': 765, 'learning_rate': 0.03553657160807668, 'max_depth': 10, 'num_leaves': 31, 'min_child_samples': 36, 'subsample': 0.77842159878613, 'colsample_bytree': 0.6100188923196663, 'reg_alpha': 0.7428920946820997, 'reg_lambda': 0.000297891702742171} because of the following error: MlflowException("API request to http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000/api/2.0/mlflow/runs/log-metric failed with exception HTTPConnectionPool(host='ec2-13-244-94-213.af-south-1.compute.amazonaws.com', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/log-metric (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x798afe84b230>: Failed to establish a new connection: [Errno 111] Connection refused'))").
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
       

MlflowException: API request to http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000/api/2.0/mlflow/runs/log-metric failed with exception HTTPConnectionPool(host='ec2-13-244-94-213.af-south-1.compute.amazonaws.com', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/log-metric (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x798afe84b230>: Failed to establish a new connection: [Errno 111] Connection refused'))